In [9]:

import pandas as pd
import numpy as np
import pyspark
from pydataset import data
from vega_datasets import data
from pyspark.sql.functions import month, year, quarter
from pyspark.sql.functions import *


In [3]:
# start spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark

22/09/12 08:33:11 WARN Utils: Your hostname, Jarads-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.27 instead (on interface en0)
22/09/12 08:33:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/12 08:33:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/12 08:33:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/12 08:33:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Create a spark data frame that contains your favorite programming languages.

The name of the column should be language
View the schema of the dataframe
Output the shape of the dataframe
Show the first 5 records in the dataframe

In [4]:
# Create 
pandas_df = pd.DataFrame({'language':['python', 'mysql', 'pandas', 'numpy', 'nosql', 'spark']})

df = spark.createDataFrame(pandas_df)

In [5]:
# View the Schema of the dataframe
df.printSchema()

root
 |-- language: string (nullable = true)



In [7]:
# show first 5 records 
df.show(5)

+--------+
|language|
+--------+
|  python|
|   mysql|
|  pandas|
|   numpy|
|   nosql|
+--------+
only showing top 5 rows



In [6]:
#shape
print((df.count(), len(df.columns)))

(6, 1)


---
Load the mpg dataset as a spark dataframe.

a. Create 1 column of output that contains a message like the one below:

The 1999 audi a4 has a 4 cylinder engine.

For each vehicle.

b. Transform the trans column so that it only contains either manual or auto.

In [11]:
from pydataset import data

In [12]:
mpg = spark.createDataFrame(data('mpg'))

mpg.head(5)

[Row(manufacturer='audi', model='a4', displ=1.8, year=1999, cyl=4, trans='auto(l5)', drv='f', cty=18, hwy=29, fl='p', class='compact'),
 Row(manufacturer='audi', model='a4', displ=1.8, year=1999, cyl=4, trans='manual(m5)', drv='f', cty=21, hwy=29, fl='p', class='compact'),
 Row(manufacturer='audi', model='a4', displ=2.0, year=2008, cyl=4, trans='manual(m6)', drv='f', cty=20, hwy=31, fl='p', class='compact'),
 Row(manufacturer='audi', model='a4', displ=2.0, year=2008, cyl=4, trans='auto(av)', drv='f', cty=21, hwy=30, fl='p', class='compact'),
 Row(manufacturer='audi', model='a4', displ=2.8, year=1999, cyl=6, trans='auto(l5)', drv='f', cty=16, hwy=26, fl='p', class='compact')]

In [13]:
# Create a column that contains message like listed above for each vehicle
mpg.select(
    concat(lit("The "), 
    mpg.year,lit(' '), 
    mpg.manufacturer, lit(' '),  
    mpg.model, lit(' has a '), 
    mpg.cyl, lit(' cylinder engine.')
    ).alias('Description')
).show(truncate=False)

+--------------------------------------------------------------+
|Description                                                   |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

In [14]:
mpg = mpg.select("*", mpg.trans, regexp_extract("trans", r"(.\w+)", 1).alias('transformed_trans'))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+-----------------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|     trans|transformed_trans|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+-----------------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|  auto(l5)|             auto|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|manual(m5)|           manual|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|manual(m6)|           manual|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|  auto(av)|             auto|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|  auto(l5)|             auto|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+-----------------+
only showing top 5 rows




3. Load the tips dataset as a spark dataframe.

In [15]:
# Load the tips dataset as a spark dataframe.
tips = spark.createDataFrame(data('tips'))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



a. What percentage of observations are smokers?
b. Create a column that contains the tip percentage
c. Calculate the average tip percentage for each combination of sex and smoker.

In [16]:
tips.groupBy('smoker').count().withColumn('percent', 
    (round((col('count')/tips.count()*100), 2))
).show()

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|  61.89|
|   Yes|   93|  38.11|
+------+-----+-------+



In [25]:
# Create a column that contains the tip percentage
tips.withColumn("tip_percentage", expr("tip / total_bill")*100).show()
# Calculate the average tip percentage for each combination of sex and smoker.
#tips.withColumn("tip_percentage", round(col("tip") / col("total_bill")*100,2)).groupby("sex").pivot("smoker").agg(round(mean("tip_percentage"), 2)).show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|    tip_percentage|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 18.62396204033215|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2| 22.80501710376283|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|11.607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|13.031914893617023|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|21.853856562922868|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 16.65043816942551|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|14.180374361883

In [27]:
tips.withColumn("tip_percentage", col("tip") / col("total_bill")*100,2).groupby("sex").pivot("smoker").agg(round(mean("tip_percentage")).show()

SyntaxError: unexpected EOF while parsing (3914014336.py, line 1)